In [23]:
import numpy as np 
import pandas as pd
from subprocess import check_output
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

# list all raw dataset
print(check_output(["ls", "../../data/raw"]).decode("utf8"))

members.csv
sample_submission_zero.csv
train.csv
transactions.csv
user_logs.csv



## 1. Manipulate 3 small tables : train, test, membership

In [2]:
# import small datasets
train = pd.read_csv("../../data/raw/train.csv")
test = pd.read_csv("../../data/raw/sample_submission_zero.csv")
member = pd.read_csv("../../data/raw/members.csv")

In [3]:
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116194 entries, 0 to 5116193
Data columns (total 7 columns):
msno                      object
city                      int64
bd                        int64
gender                    object
registered_via            int64
registration_init_time    int64
expiration_date           int64
dtypes: int64(5), object(2)
memory usage: 273.2+ MB


In [4]:
test = test.merge(member[['msno', 'registered_via']], on='msno', how='left')
train = train.merge(member[['msno', 'registered_via']], on='msno', how='left')
train.registered_via.fillna(-1) # fill -1 if registered_via is na value
test.registered_via.fillna(-1)

0         3.0
1         3.0
2         3.0
3         7.0
4         7.0
5         3.0
6         7.0
7         3.0
8         9.0
9         7.0
10        9.0
11        7.0
12        7.0
13        3.0
14        3.0
15        9.0
16        7.0
17        7.0
18        3.0
19        7.0
20        9.0
21        3.0
22        7.0
23        9.0
24        9.0
25        9.0
26        3.0
27        3.0
28       -1.0
29        3.0
         ... 
970930    7.0
970931    9.0
970932    9.0
970933    7.0
970934    9.0
970935    9.0
970936    9.0
970937    7.0
970938    7.0
970939    9.0
970940    7.0
970941    9.0
970942    9.0
970943    9.0
970944    7.0
970945    7.0
970946    9.0
970947    7.0
970948    9.0
970949    7.0
970950    7.0
970951    7.0
970952    9.0
970953    7.0
970954    7.0
970955    7.0
970956    7.0
970957    7.0
970958    9.0
970959   -1.0
Name: registered_via, dtype: float64

## 2. Manipulate 2 big tables :transaction + user_log

In [49]:
# import subset of transaction data
tran = pd.read_csv('../../data/raw/transactions.csv',nrows=3000000)
tran = tran.loc[(tran.transaction_date>=20160130) & (log.date<=20170130)]

In [51]:
tran = tran[['msno', 'is_cancel']].groupby("msno", as_index=False).sum()
tran.columns = ['msno', 'cancel_times']

In [5]:
# import subset log data
log = pd.read_csv('../../data/raw/user_logs.csv',nrows=3000000)
log.info()
print(log.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 9 columns):
msno          object
date          int64
num_25        int64
num_50        int64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(1), int64(7), object(1)
memory usage: 206.0+ MB
                                           msno      date  num_25  num_50  \
0  rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150513       0       0   
1  rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150709       9       1   
2  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150105       3       3   
3  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150306       1       0   
4  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150501       3       0   

   num_75  num_985  num_100  num_unq  total_secs  
0       0        0        1        1     280.335  
1       0        0        7       11    1658.948  
2       0        0       

My assumption : 

The recent 3-month listening experience influences the user's churn decision. So:

- For users in training set , we only extract their log records 3 months before Feburary 2017

- For users in test set , we only extract the their log records 3 months before March 2017



In [54]:
train_log = log.loc[(log.date>=20161101) & (log.date<=20170130)]
test_log = log.loc[(log.date>=20161201) & (log.date<=20170228)]

In [55]:
train_log["total_min"] = train_log.total_secs/60
test_log["total_min"] = test_log.total_secs/60

C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\dell-pc\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [56]:
train_log.total_min.describe()

count    404748.000000
mean        133.325710
std         161.907958
min           0.000133
25%          32.819129
50%          78.251475
75%         169.074150
max       12933.822783
Name: total_min, dtype: float64

In [57]:
md_train =  train_log[['msno', 'total_min']].groupby(['msno'], as_index=False).median()
md_train['log_md'] = np.log((md_train.total_min)).astype('int') 

In [58]:
md_test =  test_log[['msno', 'total_min']].groupby(['msno'], as_index=False).median()
md_test['log_md'] = np.log((md_test.total_min)).astype('int') 

## 3. merge tables together

In [65]:
train = train.merge(md_train, how='left')
train.log_md= train.log_md.fillna(0)

train = train.merge(tran,how = "left")
train.cancel_times= train.cancel_times.fillna(0)

train = train.drop('total_min', 1)

In [70]:
train.head(10)

,msno,is_churn,registered_via,log_md,cancel_times
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,9.0,0.0,0.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,9.0,0.0,0.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,9.0,0.0,1.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,9.0,0.0,0.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,9.0,0.0,0.0
5,GBy8qSz16X5iYWD+3CMxv/Hm6OPSrXBYtmbnlRtknW0=,1,9.0,0.0,1.0
6,lYLh7TdkWpIoQs3i3o6mIjLH8/IEgMWP9r7OpsLX0Vo=,1,9.0,0.0,0.0
7,T0FF6lumjKcqEO0O+tUH2ytc+Kb9EkeaLzcVUiTr1aE=,1,9.0,5.0,0.0
8,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,9.0,0.0,0.0
9,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1,9.0,0.0,0.0


In [71]:
test = test.merge(md_test, how='left')
test.log_md= test.log_md.fillna(0)

test = test.merge(tran,how = "left")
test.cancel_times= test.cancel_times.fillna(0)

test = test.drop('total_min', 1)

In [72]:
test.head(10)

,msno,is_churn,registered_via,log_md,cancel_times
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,3.0,0.0,0.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0,3.0,0.0,0.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,3.0,0.0,0.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0,7.0,0.0,0.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0,7.0,0.0,1.0
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0,3.0,4.0,0.0
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,0,7.0,0.0,0.0
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,0,3.0,4.0,0.0
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,0,9.0,4.0,0.0
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,0,7.0,4.0,0.0


In [73]:
# compute churn mean
print("churn mean")

base_mean = train.is_churn.mean()
print(base_mean)

churn mean
0.06392287077349786


In [74]:
# compute the churn mean based on different combination of registration method and mins per song
churn_mean = train[['registered_via', 'log_md','cancel_times','is_churn']].groupby(['registered_via', 'log_md','cancel_times'], as_index=False).mean()
churn_mean.columns = ['registered_via', 'log_md','cancel_times','churn_mean']
churn_mean.head(5)

,registered_via,log_md,cancel_times,churn_mean
0,3.0,-5.0,0.0,0.000000
1,3.0,-4.0,0.0,0.000000
2,3.0,-3.0,0.0,0.230769
3,3.0,-2.0,0.0,0.058824
4,3.0,-2.0,2.0,0.000000


In [75]:
# make prediction on test data
test = test.merge(churn_mean, how='left')
test.churn_mean = test.churn_mean.fillna(base_mean) # mean permutation
test.head(10)

,msno,is_churn,registered_via,log_md,cancel_times,churn_mean
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,3.0,0.0,0.0,0.132962
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0,3.0,0.0,0.0,0.132962
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0,3.0,0.0,0.0,0.132962
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0,7.0,0.0,0.0,0.029099
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0,7.0,0.0,1.0,0.183004
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0,3.0,4.0,0.0,0.107573
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,0,7.0,0.0,0.0,0.029099
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,0,3.0,4.0,0.0,0.107573
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,0,9.0,4.0,0.0,0.072314
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,0,7.0,4.0,0.0,0.018973


In [76]:
# user churn mean as the prediction of "is_churn"
test['is_churn'] = test.churn_mean

In [78]:
test[['msno','is_churn']].to_csv('2nd_submission.csv', index=False)